# 基于Transformers的命名实体识别(NER)模型

## Step1 导入相关包

In [1]:
import evaluate
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset

## Step2 加载数据集

In [2]:
ner_dataset = load_dataset("lansinuote/peoples-daily-ner")
ner_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [3]:
ner_dataset["train"]

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 20865
})

In [4]:
label_list = ner_dataset["train"].features["ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

## Step3 数据预处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base", chache_dir="./models")

In [6]:
tokenizer(ner_dataset["train"][0]["tokens"], is_split_into_words=True)

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
res = tokenizer("interesting word")
res

{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [8]:
res.word_ids()

[None, 0, 0, 0, 0, 1, None]

In [9]:
def process_function(examples):
    tokenized_examples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_examples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)  # -100 will be ignored by the loss function
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_examples["labels"] = labels
    return tokenized_examples

In [10]:
tokenized_dataset = ner_dataset.map(process_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [11]:
print(tokenized_dataset["train"][0])

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}


## Step4 创建模型

In [12]:
model = AutoModelForTokenClassification.from_pretrained("hfl/chinese-macbert-base", num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step5 创建评估函数

In [13]:
seqeval = evaluate.load("seqeval")

In [19]:
import numpy as np
def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Step6 配置训练参数

In [20]:
args = TrainingArguments(
    output_dir="./models_for_ner",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=50
)

## Step7 创建训练器

In [21]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer)
)

## Step8 模型训练

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.013800,0.021058,0.952578,0.944763,0.948655,0.994228
2,0.006100,0.021551,0.955391,0.954608,0.954999,0.994742
3,0.004200,0.021881,0.962586,0.956795,0.959682,0.995145


TrainOutput(global_step=3915, training_loss=0.008491937196688634, metrics={'train_runtime': 813.9133, 'train_samples_per_second': 76.906, 'train_steps_per_second': 4.81, 'total_flos': 3289035322521630.0, 'train_loss': 0.008491937196688634, 'epoch': 3.0})

In [23]:
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 0.02746177650988102,
 'eval_precision': 0.9505744055570398,
 'eval_recall': 0.9551677852348993,
 'eval_f1': 0.9528655597214782,
 'eval_accuracy': 0.9940477855151905,
 'eval_runtime': 19.6216,
 'eval_samples_per_second': 236.321,
 'eval_steps_per_second': 14.78,
 'epoch': 3.0}

## Step9 模型预测

In [24]:
from transformers import pipeline

In [25]:
model.config.id2label = {i: label for i, label in enumerate(label_list)}

In [26]:
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

Device set to use cuda:0


In [37]:
res = ner_pipe("李明爱北京天安门，天安门上太阳升")
res

[{'entity_group': 'PER',
  'score': 0.9996362,
  'word': '李 明 爱',
  'start': 0,
  'end': 3},
 {'entity_group': 'LOC',
  'score': 0.9997462,
  'word': '北 京',
  'start': 3,
  'end': 5},
 {'entity_group': 'LOC',
  'score': 0.9979512,
  'word': '天 安 门',
  'start': 5,
  'end': 8},
 {'entity_group': 'LOC',
  'score': 0.9990337,
  'word': '天 安 门',
  'start': 9,
  'end': 12},
 {'entity_group': 'LOC',
  'score': 0.9989405,
  'word': '太 阳',
  'start': 13,
  'end': 15}]

In [38]:
ner_result = {}
x = "小明爱北京天安门，天安门上太阳升"
for r in res:
    if r["entity_group"] not in ner_result:
        ner_result[r["entity_group"]] = []
    ner_result[r["entity_group"]].append((x[r["start"]:r["end"]]))

In [39]:
ner_result

{'PER': ['小明爱'], 'LOC': ['北京', '天安门', '天安门', '太阳']}